## 混合数据增强
## What is mixup?
该模块包含称为mixup的数据增强技术的实现。 它非常有效地使计算机视觉中的模型正规化（我们用它来花时间在一个GPU上将CIFAR10训练到94％到6分钟）。

顾名思义，混合文章的作者建议在训练集图像的混合上训练模型。 例如，假设我们正在接受CIFAR10培训。 我们采用两个图像（不一定来自同一个类）而不是为原始图像提供模型，而是对它们进行线性组合：就张量而言，我们有：

`new_image = t * image1 + (1-t) * image2`

其中t是介于0和1之间的浮点数。我们为新图像指定的目标是原始目标的相同组合：

`new_target = t * target1 + (1-t) * target2`

假设目标是单热编码的（通常在PyTorch中不是这种情况）。 就这么简单。

狗还是猫？ 这里正确的答案是70％的狗和30％的猫！

如上图所示，人眼有点难以理解以这种方式获得的图像（尽管我们确实看到了狗和猫的形状）。 然而，它在某种程度上对模型更有意义，模型更有效地训练。 一个重要的注意事项是，当使用混合训练时，最终的损失（训练或验证）将高于没有训练时的训练，即使准确性要好得多：这样训练的模型会使预测变得不那么自信。

### 基础训练
为了测试这个方法，我们首先创建一个simple_cnn并像使用basic_train一样训练它，这样我们就可以将它的结果与使用mixup训练的网络进行比较。

In [2]:
from fastai.vision import *
from fastai.basics import *
from fastai.callbacks import *

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy])

In [4]:
learn.fit(8)

epoch,train_loss,valid_loss,accuracy,time
0,0.112798,0.110776,0.961727,01:01
1,0.085694,0.067247,0.975466,01:01
2,0.063270,0.050165,0.981354,01:04
3,0.043400,0.052239,0.978901,01:02
4,0.033353,0.034974,0.988714,01:03
5,0.037377,0.033917,0.987242,01:00
6,0.027177,0.033356,0.988224,01:01
7,0.031049,0.049723,0.981354,01:02


### 库中的混合实现
在原始文章中，作者提出了四件事：
1. 创建两个独立的数据加载器，并在每次迭代时从每个数据加工器中绘制一个批次以混合它们
2. 在带有参数alpha的beta分布后绘制t的值（在他们的文章中建议为0.4）
3. 将两个批次混合使用相同的值t
4. 使用单热编码目标

该模块的实现基于这些建议，并在实验结果提出改进性能的情况下进行了修改。

作者建议使用参数α= 0.4的β分布。 （一般来说，beta分布有两个参数，但在这种情况下它们将是相同的。）为什么他们建议这样做？ 好吧，根据他们建议的参数，beta版本分布如下：

意味着选择接近0或1的值的概率非常高（在这种情况下，混合图像主要来自一个类别），然后在中间拾取某些东西的概率要小得多（注意0.33是例如，几乎与0.5一样可能。

虽然这很有效，但这不是最快的方式，这是我们调整的第一个建议。使用这种方法的不必要的减速来自于在每次迭代时绘制两个不同的批次，这意味着加载两倍的图像数量并且另外将任何其他数据增强功能应用于它们。为了避免这种情况，我们在一个批处理上使用混合版本自己应用混合版本：这样，混合的图像仍然不同。

对整个批次使用相同的t值是我们修改的另一个建议。在我们的实验中，我们注意到如果我们为批次中的每个图像绘制不同的t，则模型训练得更快。 （两个选项在准确性方面得到了相同的结果，只是一个人到达那里的速度更慢。）

最后，请注意，使用此策略，我们可能会创建重复的图像：假设我们将image0与image1混合，image1与image0混合，我们为第一个混合绘制t = 0.1，为第二个混合绘制t = 0.9。 然后

`image0 * 0.1 + shuffle0 * (1-0.1) = image0 * 0.1 + image1 * 0.9`

和

`image1 * 0.9 + shuffle1 * (1-0.9) = image1 * 0.9 + image0 * 0.1`

会是一样的。 当然，我们必须有点不走运，但实际上，当我们没有删除重复时，我们看到准确性下降。 为了避免这种情况，诀窍是替换我们绘制的t的向量：

`t = max(t, 1-t)`

两个参数相等的β分布在任何情况下都是对称的，这样我们就可以确保最大系数始终接近第一个图像（非混洗批次）。

### Adding mixup to the mix
我们现在将MixUpCallback添加到我们的学习者，以便它相应地修改我们的输入和目标。 mixup函数在幕后为我们执行此操作，以及下面描述的一些其他调整：

In [5]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy]).mixup()
learner.fit(8)

epoch,train_loss,valid_loss,accuracy,time
0,0.306032,0.129113,0.964671,01:01
1,0.273092,0.090407,0.978410,01:02
2,0.251418,0.071056,0.984298,01:02
3,0.237596,0.067254,0.986261,01:01
4,0.241754,0.060894,0.987733,01:04
5,0.230832,0.058706,0.989696,01:05
6,0.234032,0.051232,0.988714,01:03
7,0.226659,0.044110,0.993621,01:05


使用mixup进行训练可提高最佳准确度。 请注意，验证损失高于没有混淆，因为模型的预测不太自信：没有混淆，大多数预测非常接近0或1.（就概率而言）而混合模型使得预测更加细微。 在使用混音之前，请确保您知道优化更低损耗或更高精度是否更重要。

### class MixUpCallback
`MixUpCallback(learn:Learner, alpha:float=0.4, stack_x:bool=False, stack_y:bool=True) :: LearnerCallback`

回调创建混合输入和目标。

使用beta分布的参数alpha创建一个callup for mixup。 stack_x和stack_y确定我们是否使用绘制的向量lambda堆叠输入/目标或执行线性组合。 （一般情况下，当输入或输出对应于类别或类时，我们将它们堆叠起来，否则进行线性组合。）

### Callback methods
你不要自己调用 - 他们会被fastai的Callback系统自动调用，以启用该类的功能。

<b>on_batch_begin</b>

`on_batch_begin(last_input, last_target, train, **kwargs)`

如果训练，将mixup应用于last_input和last_target。

使用self.alpha绘制beta分布后的lambda向量，并根据self.stack_x和self.stack_y对last_input和last_target进行混合操作。

### Dealing with the loss
我们经常要修改损失，以便与mixup兼容。 PyTorch在可能的情况下非常小心地避免使用单热编码目标，因此撤消这一点似乎有点拖累。 对我们来说幸运的是，如果损失是经典的交叉熵，我们就有了

`loss(output, new_target) = t * loss(output, target1) + (1-t) * loss(output, target2)`

因此，我们不会对任何东西进行单一编码，而只是计算这两种损失并找到线性组合。

以下类用于调整混合的损失。 请注意，如果需要，mixup函数将使用它来更改Learner.loss_func。

<b>class MixUpLoss</b>

`MixUpLoss(*args, **kwargs) :: PrePostInitMeta :: Module`

适应损失函数暴击与混合。